In [1]:
import pandas as pd

data = pd.read_csv('jester-v1-train.csv',header = None, delimiter = ';')
labels = pd.read_csv('jester-v1-labels.csv', header = None, delimiter = ';')
labels.head(20)

,0
0,Swiping Left
1,Swiping Right
2,Swiping Down
3,Swiping Up
4,Pushing Hand Away
5,Pulling Hand In
6,Sliding Two Fingers Left
7,Sliding Two Fingers Right
8,Sliding Two Fingers Down
9,Sliding Two Fingers Up


In [2]:
swipes_train = data[(data[1] == 'Swiping Right') | (data[1] == 'Swiping Left') 
          | (data[1] == 'Swiping Up') | (data[1] == 'Swiping Down')]

swipes_train.to_csv('swipes_train.csv',index = False)
swipes_train.iloc[0,1][8:]

'Right'

In [3]:
# The functions used for reading in the images

import cv2
import os
import numpy as np



def image(folder,file):
    # First put the correct number of zeros in the image string
    fileStr = str(file)
    if len(fileStr) == 1:
        fileStr = '0000' + fileStr + '.jpg'
    else:
        fileStr = '000' + fileStr + '.jpg'
        
    location = '20bn-jester-v1//' + str(folder) + '//' + fileStr
    im = cv2.imread(location)
    return im

def grayScale(im):
    return np.dot(im,np.asarray([0.2,0.4,0.4]))

def imProcess(im):    
    gray = grayScale(im)
    return cv2.resize(gray,(176,100))

def readAndProcess(gesture_No,image_No):
    return imProcess(image(gesture_No,image_No))

# This function reads in the middle frame of a gesture

def readMiddleImage(gesture_no):
    frames = len(os.listdir('20bn-jester-v1/'+str(gesture_no)))
    return readAndProcess(gesture_no,int(frames/2))

In [4]:
# We define a function which converts labels to vectors for classification

def label2vec(label):
    result = np.zeros(4)
    direction = label[8:]
    if direction == 'Right':
        result[0] = 1
    elif direction == 'Left':
        result[1] = 1
    elif direction == 'Up':
        result[2] = 1
    elif direction == 'Down':
        result[3] =1
        
    return result



In [5]:
# Fill arrays with the data

swipes_X = np.zeros((16725,100,176))
swipes_Y = np.zeros((16725,4))

for i in range(16725):
    swipes_X[i] = readMiddleImage(swipes_train.iloc[i,0])
    swipes_Y[i] = label2vec(swipes_train.iloc[i,1])

In [6]:
import matplotlib.pyplot as plt

n = 1005
print(swipes_train.iloc[n,1])
plt.imshow(swipes_X[n])
swipes_X = swipes_X.reshape((16725,100,176,1))

Swiping Right


In [9]:
import keras
from keras.layers import Conv2D, Dense, MaxPooling2D, Activation, Dropout, Flatten
from keras.models import Sequential

model1 = Sequential()
model1.add(Conv2D(32,(3,3), padding='same', activation='relu', input_shape = swipes_X.shape[1:]))
model1.add(Conv2D(32,(3,3),padding = 'same',activation = 'relu'))
model1.add(MaxPooling2D(pool_size = (2,2)))
model1.add(Dropout(0.25))
model1.add(Flatten())
model1.add(Dense(128,activation = 'relu'))
model1.add(Dense(4,activation = 'softmax'))
model1.compile(optimizer = 'adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [11]:
model1.fit(x = swipes_X, y = swipes_Y, epochs = 10, batch_size = 25)

Epoch 1/10
16725/16725 [==============================] - 373s 22ms/step - loss: 12.0136 - acc: 0.2546
Epoch 2/10
16725/16725 [==============================] - 369s 22ms/step - loss: 12.0136 - acc: 0.2546
Epoch 3/10
16725/16725 [==============================] - 362s 22ms/step - loss: 12.0136 - acc: 0.2546
Epoch 4/10
16050/16725 [===========================>..] - ETA: 14s - loss: 12.0168 - acc: 0.2545

KeyboardInterrupt: 